# TLC Hourly Demand Processing (Optimized for Memory)
This notebook processes TLC Parquet files efficiently by:
- Processing files in chunks (avoids loading all data at once)
- Saving intermediate hourly aggregates
- Combining results and adding features
- Saving final partitioned Parquet files
- Optional cleanup of intermediate files


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import glob
import os
import holidays
import re
from tqdm import tqdm

## Step 2: Setup Directories
We create separate directories for intermediate and final outputs:
- `final_dir` for final partitioned Parquet files
- `intermediate_dir` for intermediate hourly aggregates


In [2]:
output_dir = "data/hourly_demand_partitioned"
final_dir = Path(output_dir) / "final"
intermediate_dir = Path(output_dir) / "intermediate"

final_dir.mkdir(parents=True, exist_ok=True)
intermediate_dir.mkdir(parents=True, exist_ok=True)


## Step 3: Infer Date Range from Filenames
We infer the date range from filenames to filter data later.


In [3]:
input_pattern = "data/tlc_*/**/*.parquet"
input_files = glob.glob(input_pattern, recursive=True)

dates = []
for f in input_files:
    match = re.search(r'(\d{4})[-_]?(\d{2})', f)
    if match:
        year, month = int(match.group(1)), int(match.group(2))
        dates.append(pd.Timestamp(year=year, month=month, day=1))

if dates:
    start_date = min(dates)
    end_date = max(dates) + pd.offsets.MonthEnd(1)
else:
    raise ValueError("❌ No valid dates found in filenames")

print(f"✅ Date range inferred: {start_date} to {end_date}")


✅ Date range inferred: 2023-01-01 00:00:00 to 2025-06-30 00:00:00


## Step 4: Process Files in Chunks and Save Intermediate Aggregates
We process each file individually, normalize schema, compute hourly aggregates, and save intermediate results.


In [4]:
for f in tqdm(input_files, desc="Processing files"):
    chunk = pd.read_parquet(f)

    # Normalize schema
    if 'tpep_pickup_datetime' in chunk.columns:
        chunk.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                              'tpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)
    elif 'lpep_pickup_datetime' in chunk.columns:
        chunk.rename(columns={'lpep_pickup_datetime': 'pickup_datetime',
                              'lpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)

    # Keep only necessary columns
    chunk = chunk[['pickup_datetime', 'dropoff_datetime']]

    # Convert datetime
    chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'])
    chunk['dropoff_datetime'] = pd.to_datetime(chunk['dropoff_datetime'])

    # Compute trip duration and filter invalid trips
    chunk['trip_duration_min'] = (chunk['dropoff_datetime'] - chunk['pickup_datetime']).dt.total_seconds() / 60
    chunk = chunk[chunk['trip_duration_min'] > 0]

    # Aggregate hourly demand for this file
    chunk['date_hour'] = chunk['pickup_datetime'].dt.floor('h')
    hourly = chunk.groupby('date_hour').size().reset_index(name='rides')

    # Save intermediate result
    hourly.to_parquet(intermediate_dir / f"hourly_{os.path.basename(f)}", index=False)


Processing files: 100%|██████████| 60/60 [02:47<00:00,  2.80s/it]


## Step 5: Combine Intermediate Results
We combine all intermediate hourly aggregates and filter by date range.


In [5]:
hourly_files = glob.glob(str(intermediate_dir / "hourly_*.parquet"))
hourly_demand = pd.concat([pd.read_parquet(f) for f in hourly_files], ignore_index=True)

# Aggregate again to merge overlapping hours
hourly_demand = hourly_demand.groupby('date_hour', as_index=False)['rides'].sum()

# Filter by inferred date range
hourly_demand = hourly_demand[(hourly_demand['date_hour'] >= start_date) & (hourly_demand['date_hour'] <= end_date)]


## Step 6-9: Add Features (Time-based, Lag, Holiday, Weather)
We add additional features for modeling and analysis.


In [6]:
# Time-based features
hourly_demand['year'] = hourly_demand['date_hour'].dt.year.astype('int16')
hourly_demand['month'] = hourly_demand['date_hour'].dt.month.astype('int8')
hourly_demand['hour'] = hourly_demand['date_hour'].dt.hour.astype('int8')
hourly_demand['weekday'] = hourly_demand['date_hour'].dt.weekday.astype('int8')

# Lag features
for lag in [1, 2, 24]:
    hourly_demand[f'lag_{lag}'] = hourly_demand['rides'].shift(lag)

hourly_demand.dropna(inplace=True)

# Holiday flag
years = hourly_demand['year'].unique()
us_holidays = holidays.US(years=years)
hourly_demand['is_holiday'] = hourly_demand['date_hour'].dt.date.astype(str).isin(us_holidays).astype('int8')

# Merge weather data if available
weather_files = glob.glob("data/weather*.parquet")
if weather_files:
    weather = pd.concat([pd.read_parquet(f) for f in weather_files], ignore_index=True)
    weather['date_hour'] = pd.to_datetime(weather['datetime']).dt.floor('h')
    weather = weather[(weather['date_hour'] >= start_date) & (weather['date_hour'] <= end_date)]
    expected_cols = ['temp', 'precip']
    available_cols = [col for col in expected_cols if col in weather.columns]
    hourly_demand = hourly_demand.merge(weather[['date_hour'] + available_cols], on='date_hour', how='left')
    for col in available_cols:
        hourly_demand[col] = hourly_demand[col].fillna(method='ffill')
else:
    hourly_demand['temp'] = np.nan
    hourly_demand['precip'] = np.nan


## Step 10: Save Final Partitioned Parquet Files
We save the final dataset partitioned by year and month.


In [7]:
hourly_demand.iloc[:-1, :].to_parquet(final_dir, engine="pyarrow", partition_cols=['year', 'month'], index=False)
print(f"✅ Processing complete. Final data saved to {final_dir}")


✅ Processing complete. Final data saved to data\hourly_demand_partitioned\final


## Step 11: Optional Cleanup of Intermediate Files
Set `cleanup_intermediate = True` to remove intermediate files after processing.


In [8]:
cleanup_intermediate = False  # Set to False to keep intermediate files
if cleanup_intermediate:
    for f in hourly_files:
        os.remove(f)
    print(f"🧹 Intermediate files cleaned up from {intermediate_dir}")
